<a href="https://colab.research.google.com/github/flora0110/sentence_prediction/blob/main/QA_bge_m3_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [6]:
!pip install udicOpenData

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for udicOpenData: filename=udicOpenData-2.4-py3-none-any.whl size=18094423 sha256=73f8ccc16f7bf3f691c43db361c902b289d95de1767ac2c0ac69eebd9a33d4ea
  Stored in directory: /root/.cache/pip/wheels/86/4f/46/2a28bc28485ba1f9d67dead62ac6aba14089a6cf1d5fbad365
Successfully built udicOpenData


In [7]:
!pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 5.5 MB/s eta 0:00:00


In [8]:
!pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 19.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_c

In [9]:
import numpy as np
import json
from opencc import OpenCC
from sklearn.neighbors import NearestNeighbors
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [10]:
from udicOpenData.stopwords import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [11]:
from FlagEmbedding import BGEM3FlagModel

# Data import

In [2]:
folder_path = "/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sentence_predict_dataset/"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
with open(folder_path+'summary_predict/train/summary_train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(folder_path+'summary_predict/test/summary_test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [24]:
def extract_key_contents(data):
    facts = []
    sentences = []
    for item in data:
        facts.append(item.get('contents'))
        sentences.append(item.get('sentance'))
    return facts, sentences

In [14]:
train_facts, train_sentences = extract_key_contents(train_data)
test_facts, test_sentences = extract_key_contents(test_data)

In [17]:
train_fact_len = []
test_fact_len = []

for data in train_data:
    train_fact_len.append(len(data['fact_summary']))

for data in test_data:
    test_fact_len.append(len(data['fact_summary']))


In [18]:
print("===Train Dataset===")
print("Length: ", len(train_fact_len))
print("Max Length: ", max(train_fact_len))
print("Min Length: ", min(train_fact_len))
print("Avg. Length: ", sum(train_fact_len) / len(train_fact_len))

print("===Test Dataset===")
print("Length: ", len(test_fact_len))
print("Max Length: ", max(test_fact_len))
print("Min Length: ", min(test_fact_len))
print("Avg. Length: ", sum(test_fact_len) / len(test_fact_len))

===Train Dataset===
Length:  500
Max Length:  1649
Min Length:  4
Avg. Length:  464.872
===Test Dataset===
Length:  100
Max Length:  1456
Min Length:  4
Avg. Length:  460.58


In [20]:
with open(folder_path+'QA_predict/train/QA_train.json', 'r', encoding='utf-8') as f:
    train_data_QA = json.load(f)

with open(folder_path+'QA_predict/test/QA_test.json', 'r', encoding='utf-8') as f:
    test_data_QA = json.load(f)

In [25]:
train_facts_QA, train_sentences_QA = extract_key_contents(train_data_QA)
test_facts_QA, test_sentences_QA = extract_key_contents(test_data_QA)

In [26]:
train_facts_QA[0]

'1. 偽造誰的什麼文書：被告等人共同偽造了陳玉婉的戶籍謄本。\n2. 有什麼偽造行為：被告等人共同辦理假結婚，使公務員登載不實事項於戶籍登記資料內，並核發戶籍謄本。\n3. 損害他人利益的地方：被告與陳玉婉共同持登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期，導致主管機關對外僑居留管理的正確性受損。'

In [27]:
train_fact_len = []
test_fact_len = []

for data in train_data_QA:
    train_fact_len.append(len(data['contents']))

for data in test_data_QA:
    test_fact_len.append(len(data['contents']))


In [28]:
print("===Train Dataset===")
print("Length: ", len(train_fact_len))
print("Max Length: ", max(train_fact_len))
print("Min Length: ", min(train_fact_len))
print("Avg. Length: ", sum(train_fact_len) / len(train_fact_len))

print("===Test Dataset===")
print("Length: ", len(test_fact_len))
print("Max Length: ", max(test_fact_len))
print("Min Length: ", min(test_fact_len))
print("Avg. Length: ", sum(test_fact_len) / len(test_fact_len))

===Train Dataset===
Length:  500
Max Length:  6603
Min Length:  66
Avg. Length:  168.36
===Test Dataset===
Length:  100
Max Length:  6771
Min Length:  76
Avg. Length:  221.68


# Preprocessing

## 斷詞

In [29]:
cc = OpenCC('t2s')

In [30]:
# 分詞加空白
def split_passage(passage):
    word_and_tag = list(rmsw(passage, flag=True))
    _temp = list(filter(lambda x: x[1]!='nr' and x[1]!='ns', word_and_tag))
    # print(_temp)
    text = " ".join(list(map(lambda x:x[0], _temp)))
    return text

In [31]:
tokenized_train_QA = []

for data in train_facts_QA:
    tokenized_train_QA.append(split_passage(cc.convert(data)))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.956 seconds.
DEBUG:jieba:Loading model cost 0.956 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [32]:
tokenized_test_QA = []

for data in test_facts_QA:
    tokenized_test_QA.append(split_passage(cc.convert(data)))

In [33]:
def save_string_array_to_file(string_array, file_path):
    try:
        with open(file_path, 'w') as file:
            for string in string_array:
                file.write(string + '\n')
        print(f"String array saved to {file_path} successfully.")
    except Exception as e:
        print(f"Error occurred while saving string array: {e}")

In [34]:
save_string_array_to_file(tokenized_train_QA, folder_path+'tokenized_train_QA.txt')
save_string_array_to_file(tokenized_test_QA, folder_path+'tokenized_test_QA.txt')

String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sentence_predict_dataset/tokenized_train_QA.txt successfully.
String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sentence_predict_dataset/tokenized_test_QA.txt successfully.


In [ ]:
# tokenized_train_summaries = read_string_array_from_file('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt')
# tokenized_test_summaries = read_string_array_from_file('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt')

String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt successfully.
String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt successfully.


## Data embedding

In [35]:
model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

In [36]:
sword_1 = ["老师", "商品"]
sword_2 = ["教师", "货物"]

### 測試在中文上的效果

In [37]:
embeddings_1 = model.encode(sword_1,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sword_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

[[0.8509769  0.5041799 ]
 [0.464958   0.74887514]]


In [38]:
output_1 = model.encode(sword_1, return_dense=True, return_sparse=True, return_colbert_vecs=True)
output_2 = model.encode(sword_2, return_dense=True, return_sparse=True, return_colbert_vecs=True)

print(model.colbert_score(output_1['colbert_vecs'][0], output_2['colbert_vecs'][0]))
print(model.colbert_score(output_1['colbert_vecs'][0], output_2['colbert_vecs'][1]))
print(model.colbert_score(output_1['colbert_vecs'][1], output_2['colbert_vecs'][0]))
print(model.colbert_score(output_1['colbert_vecs'][1], output_2['colbert_vecs'][1]))

tensor(0.8653)
tensor(0.5597)
tensor(0.5299)
tensor(0.7482)


### train test data embedding

In [39]:
train_embeddings_all = model.encode(tokenized_train_QA, max_length=1500)

Inference Embeddings: 100%|██████████| 42/42 [26:10<00:00, 37.40s/it]


In [40]:
test_embeddings_all = model.encode(tokenized_test_QA, max_length=1500)

In [41]:
with open(folder_path + 'train_embeddings_all_QA.pkl', 'wb') as f:
    pickle.dump(train_embeddings_all, f)

In [42]:
with open(folder_path + 'test_embeddings_all_QA.pkl', 'wb') as f:
    pickle.dump(test_embeddings_all, f)

In [ ]:
# with open(folder_path + 'train_embeddings_all.pkl', 'rb') as f:
#     train_embeddings_all = pickle.load(f)

In [ ]:
# with open(folder_path + 'test_embeddings_all.pkl', 'rb') as f:
#     test_embeddings_all = pickle.load(f)

## dense - KNN

In [43]:
train_embeddings_dense = train_embeddings_all['dense_vecs']
test_embeddings_dense = test_embeddings_all['dense_vecs']

In [44]:
k = 5
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings_dense)

NearestNeighbors(metric='cosine')

In [45]:
# 將模型保存到文件
with open(folder_path + 'knn_model_dense_QA.pkl', 'wb') as f:
    pickle.dump(knn_model_SBERT, f)

In [ ]:
# # 從文件加載模型
# with open(folder_path + 'knn_model_dense.pkl', 'rb') as f:
#     knn_model_SBERT_loaded = pickle.load(f)

### 以第一篇test fact進行測試

In [46]:
distances, indices = knn_model_SBERT.kneighbors(test_embeddings_dense[0].reshape(1, -1))

In [47]:
indices

array([[143, 400,  31, 136, 129]])

In [48]:
distances

array([[0.25774252, 0.26479715, 0.2656963 , 0.26598203, 0.27219766]],
      dtype=float32)

In [49]:
test_facts[0]

' 事實摘要：被告徐鴻光（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決處有期徒刑3月、緩刑3年確定）和被告穆立明（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決無罪確定）共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡。被告穆立明偽造「志朗交通有限公司」及負責人「陳健志」（正確姓名為陳建志，誤刻為「陳健志」）印章各1枚，並偽造「陳健志」代表志朗公司與被告徐鴻光簽署的「臺北市計程車客運駕駛人自備車輛參與經營契約書」。他們以偽造的契約書向宏帝當鋪（現更名為渣打當舖）借款新臺幣（下同）10萬元，並約定由被告徐樂仁（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決無罪確定）擔任連帶保證人。宏帝當鋪誤信該車輛為被告徐鴻光所有，而陷於錯誤，交付借款。被告徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，宏帝當鋪始查知上開車輛非被告徐鴻光所有。'

In [50]:
test_facts_QA[0]

'1. 偽造的文書：「志朗交通有限公司」及負責人「陳健志」（正確姓名為陳建志，誤刻為「陳健志”）印章和偽造的契約書\n2. 偽造行為：被告徐鴻光和穆立明共同以偽造的印章和契約書向宏帝當鋪（現更名為渣打當舖）借款10萬元。\n3. 損害他人利益的地方：宏帝當鋪（現更名為渣打當舖）誤信該車輛為被告徐鴻光所有，而陷於錯誤，交付借款。被告徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，當鋪始查知上開車輛非被告徐鴻光所有。'

In [52]:
indices[0]

array([143, 400,  31, 136, 129])

In [51]:
# 打印最相近的五篇文章的索引及距離
for i, index in enumerate(indices[0]):
    print("Article Index:", index, "Distance:", distances[0][i])
    print("Article Text:", train_facts_QA[index])

Article Index: 143 Distance: 0.25774252


IndexError: list index out of range

### 使用全部test facts在train facts中尋找最相近的五篇facts並計算預測刑期

In [53]:
predicted_sentences_all = []

for test_embedding in test_embeddings_dense:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / 5

    predicted_sentences_all.append(predicted_sentence)



In [54]:
print(predicted_sentences_all)
print(test_sentences)

[205.0, 235.0, 183.0, 145.6, 150.0, 188.0, 102.0, 153.0, 171.0, 174.0, 217.0, 189.0, 168.0, 105.0, 165.0, 115.0, 228.0, 135.0, 203.0, 173.0, 241.6, 180.0, 284.0, 168.4, 282.0, 122.0, 146.0, 108.0, 150.0, 152.0, 210.4, 117.0, 135.0, 259.6, 189.0, 321.6, 120.0, 195.0, 126.0, 241.0, 130.0, 99.0, 144.0, 271.8, 226.0, 195.0, 162.0, 162.0, 123.0, 183.0, 314.2, 150.0, 212.0, 144.0, 111.0, 153.0, 362.2, 347.2, 177.0, 135.0, 253.0, 156.0, 330.0, 143.0, 96.0, 137.6, 170.0, 290.6, 270.4, 120.0, 170.0, 277.8, 132.0, 114.0, 247.0, 241.0, 159.0, 200.0, 166.4, 234.4, 286.0, 204.0, 144.0, 150.0, 273.4, 118.4, 186.0, 268.0, 133.4, 187.0, 154.0, 123.0, 83.0, 162.0, 198.0, 428.8, 300.2, 156.0, 117.0, 92.0]
[150, 242, 60, 302, 365, 150, 90, 120, 120, 180, 120, 90, 120, 90, 90, 90, 180, 270, 425, 50, 150, 90, 365, 195, 150, 50, 345, 150, 150, 90, 425, 135, 120, 240, 20, 240, 60, 120, 60, 150, 90, 240, 365, 210, 150, 120, 32, 150, 505, 90, 90, 120, 120, 120, 165, 180, 302, 90, 120, 730, 240, 40, 180, 90, 90

In [55]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)

In [56]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 19523.793200000004
Linear Regression - RMSE: 139.72756778817845
Linear Regression - MAE: 95.55


## colbert

In [57]:
train_embeddings_colbert = train_embeddings_all['colbert_vecs']
test_embeddings_colbert = test_embeddings_all['colbert_vecs']

In [58]:
max_length_train = max(len(vec) for vec in train_embeddings_colbert)
max_length_test = max(len(vec) for vec in test_embeddings_colbert)
max_length = max(max_length_train, max_length_test)
max_length

TypeError: 'NoneType' object is not iterable

In [59]:
def colbert_metric(vector1, vector2):
    # 将一维向量重塑为二维数组
    vector1_reshaped = vector1.reshape(max_length, 1024)
    vector2_reshaped = vector2.reshape(max_length, 1024)

    # 计算相似度
    colbert_similarity = model.colbert_score(vector1_reshaped, vector2_reshaped)
    # 计算距离（1 - 相似度）
    distance = 1 - colbert_similarity
    # 返回距离值
    return distance  # 这里使用平均值作为最终的距离值，您可以根据需要选择其他方法

In [60]:
k = 5
knn_model_colbert = NearestNeighbors(n_neighbors=k, metric=colbert_metric)

In [61]:
padded_train_embeddings_colbert = []
for embedding in train_embeddings_colbert:
  zero_embedding = np.zeros((max_length, 1024))
  for i, vec in enumerate(embedding):
    zero_embedding[i] = vec
  padded_train_embeddings_colbert.append(zero_embedding)

TypeError: 'NoneType' object is not iterable

In [ ]:
reshape_padded_train_embeddings_colbert = np.array(padded_train_embeddings_colbert).reshape(500,-1)

In [ ]:
reshape_padded_train_embeddings_colbert.shape

In [ ]:
knn_model_colbert.fit(reshape_padded_train_embeddings_colbert)

In [ ]:
padded_test_embeddings_colbert = []
for embedding in test_embeddings_colbert:
  zero_embedding = np.zeros((max_length, 1024))
  for i, vec in enumerate(embedding):
    zero_embedding[i] = vec
  padded_test_embeddings_colbert.append(zero_embedding)

In [ ]:
reshape_padded_test_embeddings_colbert = np.array(padded_test_embeddings_colbert).reshape(100,-1)

In [ ]:
distances, indices = knn_model_colbert.kneighbors(reshape_padded_test_embeddings_colbert)

In [ ]:
distances

In [ ]:
indices

In [ ]:
predicted_sentences_all = []
for nearest_articles_indices in indices:
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    predicted_sentences = 0
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / 5

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)

# 平均

In [ ]:
avg_sentence = sum(train_sentences)/len(train_sentences)

In [ ]:
avg_sentence

187.724

In [ ]:
avg_sentences = [avg_sentence]*len(test_sentences)

In [ ]:
mse_linear = mean_squared_error(test_sentences, avg_sentences)
rmse_linear = mean_squared_error(test_sentences, avg_sentences, squared=False)
mae_linear = mean_absolute_error(test_sentences, avg_sentences)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16325.817136000003
Linear Regression - RMSE: 127.77252105206347
Linear Regression - MAE: 89.10199999999999
